In [11]:
import pandas as pd
import sqlite3
import warnings
warnings.filterwarnings("ignore")

In [12]:
# First step is to get dataframe by calling read_csv function of pandas
data=pd.read_csv('Uncleaned_employees_final_dataset.csv')

In [13]:
# Now we should check null values in our dataframe
data.isnull().sum()

employee_id                 0
department                  0
region                      0
education                 771
gender                      0
recruitment_channel         0
no_of_trainings             0
age                         0
previous_year_rating     1363
length_of_service           0
KPIs_met_more_than_80       0
awards_won                  0
avg_training_score          0
dtype: int64

In [14]:
# As we have null values in 2 columns , So we will remove those records from our dataframe
data.dropna(inplace=True)

In [15]:
# Now we should remove duplicate records from our dataframe
data.drop_duplicates(keep="first",inplace=True)

In [16]:
# Now we will check if each column has relevant data type or not
data.dtypes

employee_id                int64
department                object
region                    object
education                 object
gender                    object
recruitment_channel       object
no_of_trainings            int64
age                        int64
previous_year_rating     float64
length_of_service          int64
KPIs_met_more_than_80      int64
awards_won                 int64
avg_training_score         int64
dtype: object

In [17]:
# Here we can see that one column called previous_year_rating is in floating data type , while it should be integer.
# So we will convert this floating data type into integer64.

In [18]:
data["previous_year_rating"]=data["previous_year_rating"].astype("int64")

In [19]:
# Now we will save this cleaned dataframe as "Cleaned_employees_final_dataset.csv" in a csv file format.

In [20]:
data.to_csv("Cleaned_employees_final_dataset.csv",index=False)

In [21]:
# Now we will load our cleaned csv dataset into a new dataframe.

data1 = pd.read_csv("Cleaned_employees_final_dataset.csv")

# We will Create a SQLite in-memory database and establish a connection
conn = sqlite3.connect(':memory:')

# Store the DataFrame in a SQL table
data1.to_sql('employee', conn, index=False)

15422

# Task 1
Find the average age of employees in each department and gender group.
(Round average age up to two decimal places if needed).

In [54]:
query = "select department,gender,round(avg(age),2) as average_age from employee group by department,gender"
result = pd.read_sql_query(query, conn)

In [55]:
print(result)

           department gender  average_age
0           Analytics      f        29.88
1           Analytics      m        33.54
2             Finance      f        33.33
3             Finance      m        33.15
4                  HR      f        35.57
5                  HR      m        34.98
6               Legal      f        33.72
7               Legal      m        34.69
8          Operations      f        36.37
9          Operations      m        36.88
10        Procurement      f        36.86
11        Procurement      m        36.73
12                R&D      f        31.69
13                R&D      m        33.40
14  Sales & Marketing      f        34.90
15  Sales & Marketing      m        35.90
16         Technology      f        35.76
17         Technology      m        35.87


# Task 2
List the top 3 departments with the highest average training scores.
(Round average age up to two decimal places if needed).

In [56]:
query = "select department,round(avg(avg_training_score),2)as avg_training_score from employee group by department order by avg_training_score desc limit 3"
result = pd.read_sql_query(query, conn)

In [57]:
print(result)

   department  avg_training_score
0   Analytics               84.59
1         R&D               84.42
2  Technology               79.81


# Task 3
Find the percentage of employees who have won awards in each region.
(Round average age up to two decimal places if needed).

In [60]:
query = "WITH CTE(REGION,TOTAL) AS (SELECT REGION,COUNT(*) FROM employee WHERE AWARDS_WON=1 GROUP BY REGION) SELECT REGION,ROUND(TOTAL*100/(SELECT COUNT(*) FROM employee WHERE AWARDS_WON=1),2) AS AWARDS_PERCENTAGE FROM CTE"
result = pd.read_sql_query(query, conn)

In [61]:
print(result)

       REGION  AWARDS_PERCENTAGE
0   region_10                2.0
1   region_11                1.0
2   region_12                0.0
3   region_13                4.0
4   region_14                1.0
5   region_15                6.0
6   region_16                1.0
7   region_17                3.0
8   region_19                0.0
9    region_2               20.0
10  region_20                1.0
11  region_21                0.0
12  region_22                9.0
13  region_23                2.0
14  region_24                0.0
15  region_25                1.0
16  region_26                2.0
17  region_27                5.0
18  region_28                1.0
19  region_29                2.0
20   region_3                1.0
21  region_30                0.0
22  region_31                2.0
23  region_32                1.0
24  region_33                0.0
25  region_34                0.0
26   region_4                4.0
27   region_5                1.0
28   region_6                2.0
29   regio

# Task 4
Show the number of employees who have met more than 80% of KPIs for each recruitment channel and education level.

In [66]:
query = "select recruitment_channel,education,count(employee_id)AS total_employee from employee where `KPIs_met_more_than_80` =1 group by recruitment_channel,education"

result = pd.read_sql_query(query, conn)

In [67]:
print(result)

  recruitment_channel        education  total_employee
0               other        Bachelors            2044
1               other  Below Secondary              38
2               other  Masters & above            1003
3            referred        Bachelors             125
4            referred  Below Secondary               2
5            referred  Masters & above              33
6            sourcing        Bachelors            1574
7            sourcing  Below Secondary              34
8            sourcing  Masters & above             723


# Task 5
Find the average length of service for employees in each department, considering only employees with previous year ratings greater than or equal to 4.
(Round average age up to two decimal places if needed).

In [68]:
query = "select department,round(avg(length_of_service),2)as average_length_of_service from employee where previous_year_rating>= 4 group by department"

result = pd.read_sql_query(query, conn)

In [69]:
print(result)

          department  average_length_of_service
0          Analytics                       5.53
1            Finance                       5.70
2                 HR                       6.07
3              Legal                       4.79
4         Operations                       6.83
5        Procurement                       6.63
6                R&D                       4.86
7  Sales & Marketing                       6.33
8         Technology                       6.31


# Task 6
List the top 5 regions with the highest average previous year ratings.
(Round average age up to two decimal places if needed).

In [70]:
query = "select region,round(avg(previous_year_rating),2) average_previous_year_rating from employee group by region order by average_previous_year_rating desc limit 5"

result = pd.read_sql_query(query, conn)

In [71]:
print(result)

      region  average_previous_year_rating
0  region_25                          3.58
1   region_4                          3.53
2   region_8                          3.50
3  region_28                          3.49
4  region_23                          3.47


# Task 7
List the departments with more than 100 employees having a length of service greater than 5 years.

In [72]:
query = "select department,count(*)as total from employee where length_of_service>5 group by department having total>100"

result = pd.read_sql_query(query, conn)

In [73]:
print(result)

          department  total
0          Analytics    591
1            Finance    300
2                 HR    332
3         Operations   1720
4        Procurement   1023
5                R&D    115
6  Sales & Marketing   2204
7         Technology    896


# Task 8
Show the average length of service for employees who have attended more than 3 trainings, grouped by department and gender.
(Round average age up to two decimal places if needed).

In [74]:
query = "select department,gender,round(avg(length_of_service),2)as average_length_of_service from employee where no_of_trainings>3 group by department,gender"

result = pd.read_sql_query(query, conn)

In [75]:
print(result)

           department gender  average_length_of_service
0           Analytics      m                       4.80
1             Finance      m                       3.50
2          Operations      f                       7.50
3          Operations      m                       4.25
4         Procurement      f                       4.80
5         Procurement      m                       5.92
6                 R&D      m                       3.71
7   Sales & Marketing      f                       5.00
8   Sales & Marketing      m                       5.44
9          Technology      f                       6.50
10         Technology      m                       7.85


# Task 9
Find the percentage of female employees who have won awards, per department. Also show the number of female employees who won awards and total female employees.
(Round average age up to two decimal places if needed).

In [98]:
query = "select department, round(sum(case when awards_won=1 then 1 else 0 end)/count(*)*100,2) as percentage_of_female_employees_won_awards,sum(case when awards_won=1 then 1 else 0 end) as female_employees_won_awards,count(*) as total_female_employees from employee where gender='f' group by department"
result = pd.read_sql_query(query, conn)

In [99]:
print(result)

          department  percentage_of_female_employees_won_awards  \
0          Analytics                                        0.0   
1            Finance                                        0.0   
2                 HR                                        0.0   
3              Legal                                        0.0   
4         Operations                                        0.0   
5        Procurement                                        0.0   
6                R&D                                        0.0   
7  Sales & Marketing                                        0.0   
8         Technology                                        0.0   

   female_employees_won_awards  total_female_employees  
0                            3                     131  
1                            4                     180  
2                            4                     307  
3                            1                      25  
4                           42              

# Task 10
Calculate the percentage of employees per department who have a length of service between 5 and 10 years.
(Round average age up to two decimal places if needed).

In [102]:
query = "with cte as (select department,count(*) as a from employee group by department) select e.department,round(count(*)*100/(cte.a),2)as employee_percent_per_dpt from employee e left join cte on cte.department=e.department where length_of_service between 5 and 10 group by e.department"

result = pd.read_sql_query(query, conn)

In [103]:
print(result)

          department  employee_percent_per_dpt
0          Analytics                      47.0
1            Finance                      47.0
2                 HR                      45.0
3              Legal                      37.0
4         Operations                      48.0
5        Procurement                      48.0
6                R&D                      46.0
7  Sales & Marketing                      48.0
8         Technology                      45.0


# Task 11
Find the top 3 regions with the highest number of employees who have met more than 80% of their KPIs and received at least one award, grouped by department and region.

In [104]:
query = "select department,region,count(*) as total_employees from employee where KPIs_met_more_than_80 and awards_won >=1 group by department,region order by total_employees desc limit 3"

result = pd.read_sql_query(query, conn)

In [105]:
print(result)

          department    region  total_employees
0  Sales & Marketing  region_2               19
1        Procurement  region_2               13
2  Sales & Marketing  region_7               10


# Task 12
Calculate the average length of service for employees per education level and gender, considering only those employees who have completed more than 2 trainings and have an average training score greater than 75
(Round average age up to two decimal places if needed).

In [106]:
query = "select education,gender,round(avg(length_of_service),2) as average_service from employee where no_of_trainings > 2 and avg_training_score > 75 group by education,gender"

result = pd.read_sql_query(query, conn)

In [107]:
print(result)

         education gender  average_service
0        Bachelors      f             5.71
1        Bachelors      m             4.53
2  Below Secondary      m             1.67
3  Masters & above      f             6.00
4  Masters & above      m             6.70


# Task 13
For each department and recruitment channel, find the total number of employees who have met more than 80% of their KPIs, have a previous_year_rating of 5, and have a length of service greater than 10 years.

In [108]:
query = "select department,recruitment_channel,count(*) as total_employees from employee where KPIs_met_more_than_80 =1 and previous_year_rating = 5 and length_of_service > 10 group by department,recruitment_channel"

result = pd.read_sql_query(query, conn)

In [109]:
print(result)

           department recruitment_channel  total_employees
0           Analytics               other               17
1           Analytics            sourcing                3
2             Finance               other                3
3             Finance            sourcing                4
4                  HR               other                4
5                  HR            referred                1
6                  HR            sourcing                3
7               Legal               other                2
8               Legal            sourcing                2
9          Operations               other               55
10         Operations            sourcing               39
11        Procurement               other               24
12        Procurement            sourcing               14
13                R&D               other                1
14                R&D            sourcing                1
15  Sales & Marketing               other               

# Task 14
Calculate the percentage of employees in each department who have received awards, have a previous_year_rating of 4 or 5, and an average training score above 70, grouped by department and gender
(Round average age up to two decimal places if needed).

In [110]:
query = "with table1 as (select department,gender,count(*) as total2 from employee group by department,gender), table2 as (select department,gender,count(*) as total1 from employee where previous_year_rating in (4,5) and avg_training_score > 70 and awards_won = 1 group by department,gender) select table1.department,table1.gender,ifnull(round(table2.total1 * 100 / (table1.total2),2),0) as percentage_of_employees from table1 left join table2 on table1.department=table2.department and table1.gender=table2.gender"

result = pd.read_sql_query(query, conn)

In [111]:
print(result)

           department gender  percentage_of_employees
0           Analytics      f                      1.0
1           Analytics      m                      1.0
2             Finance      f                      1.0
3             Finance      m                      0.0
4                  HR      f                      0.0
5                  HR      m                      0.0
6               Legal      f                      0.0
7               Legal      m                      0.0
8          Operations      f                      0.0
9          Operations      m                      0.0
10        Procurement      f                      0.0
11        Procurement      m                      0.0
12                R&D      f                      0.0
13                R&D      m                      1.0
14  Sales & Marketing      f                      0.0
15  Sales & Marketing      m                      0.0
16         Technology      f                      1.0
17         Technology      m

# Task 15
List the top 5 recruitment channels with the highest average length of service for employees who have met more than 80% of their KPIs, have a previous_year_rating of 5, and an age between 25 and 45 years, grouped by department and recruitment channel.
(Round average age up to two decimal places if needed).

In [112]:
query = "select department,recruitment_channel,round(avg(length_of_service),2)as avg_length_service from employee where KPIs_met_more_than_80=1 and previous_year_rating=5 and age between 25 and 45 group by department,recruitment_channel order by avg_length_service desc limit 5"

result = pd.read_sql_query(query, conn)

In [114]:
print(result)

          department recruitment_channel  avg_length_service
0         Operations            referred                6.20
1         Operations               other                6.08
2         Operations            sourcing                5.93
3  Sales & Marketing               other                5.77
4        Procurement            sourcing                5.65
